In [1]:
import numpy as np
from scipy.optimize import fsolve, fixed_point
from matplotlib import pyplot as plt
import pyblp
from tqdm.notebook import trange
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas as pd

In [2]:
RNG_SEED = 8476263

rng = np.random.default_rng(RNG_SEED) # this random seeding is for reproducibility

In [3]:
# I am horrified that we have to overrun the default collinearity checks
# however, wired and satellite dummy variables are collinear
# so to prevent PyBLP from throwing a fit, we must do this.
pyblp.options.collinear_rtol = 0
pyblp.options.collinear_atol = 0

In [4]:
# fixed parameter definitions

beta1 = 1
alpha = -2
gamma0 = 1/2
gamma1 = 1/4
beta2_bar = 4
beta3_bar = 4
sigma2 = 1
sigma3 = 1

# markets and goods
T = 600
J = 4

In [5]:
# 3.1

# x_jt, w_jt are absolute value of iid standard normal variables
x = np.absolute(rng.standard_normal(size=(J,T)))
w = np.absolute(rng.standard_normal(size=(J,T)))

unobservable_mean = [0,0]
unobservable_cov = [[1,0.25],[0.25,1]]
unobservables = rng.multivariate_normal(unobservable_mean, unobservable_cov, size=(J,T))
xi = unobservables[:,:,0]
omega = unobservables[:,:,1]

In [6]:
# 3.2a
# defining the market share

def own_mkt_share_derivative(t, p, beta2, beta3): 
    # p should be a length J vector
    # betas should be num_sims
    
    u_t = np.tile(x[:,t] + xi[:,t] + alpha*p, (len(beta2), 1)) # num_sims x J 
    for j in range(J):
        if j < 2:
            u_t[:,j] = u_t[:,j] + beta2
        else:
            u_t[:,j] = u_t[:,j] + beta3
            
    Z = np.tile( 1 + np.sum(np.exp(u_t),axis=-1), (J,1)).T
    numerator = alpha*np.exp(u_t)*Z - alpha*np.square(np.exp(u_t)) # num_sims x J
    denominator = np.square(Z)
    
    return np.mean(numerator / denominator, axis=0)

def outside_mkt_share_derivative(t, p, beta2, beta3): 
    # p should be a length J vector
    # betas should be num_sims
    
    u_t = np.tile(x[:,t] + xi[:,t] + alpha*p, (len(beta2), 1)) # num_sims x J 
    for j in range(J):
        if j < 2:
            u_t[:,j] = u_t[:,j] + beta2
        else:
            u_t[:,j] = u_t[:,j] + beta3
            
    Z = np.tile( 1 + np.sum(np.exp(u_t),axis=-1), (J,1)).T
    numerator = -1*alpha*np.exp(u_t) # num_sims x J
    denominator = np.square(Z)
    
    return np.mean(numerator / denominator, axis=0)  
    
def full_mkt_share_derivative(t, p, beta2, beta3):
    # p should be a length J vector
    # betas should be num_sims
    
    u_t = np.tile(x[:,t] + xi[:,t] + alpha*p, (len(beta2), 1)) # num_sims x J 
    for j in range(J):
        if j < 2:
            u_t[:,j] = u_t[:,j] + beta2
        else:
            u_t[:,j] = u_t[:,j] + beta3
            
    Z = np.tile( 1 + np.sum(np.exp(u_t),axis=-1), (J,1)).T # num_sims x J
    
    derivatives = np.zeros((J,J))
    
    own_numerator = alpha*np.exp(u_t)*Z - alpha*np.square(np.exp(u_t)) # num_sims x J
    denominator = np.square(Z)
    
    for j in range(J):
        derivatives[j,j] = np.mean(own_numerator / denominator, axis=0)[j]
        
    for j in range(J):
        for k in range(J):
            if not (j == k):
                derivatives[j,k] = np.mean(-1*alpha*np.exp(u_t)[:,k]*np.exp(u_t)[:,j] / np.square(1 + np.sum(np.exp(u_t),axis=-1)))
    
    return derivatives

In [7]:
# s_jt(p) 
def mkt_share(t, p, beta2, beta3):
    # p should be a length J vector
    # betas should be num_sims
    
    u_t = np.tile(x[:,t] + xi[:,t] + alpha*p, (len(beta2), 1)) # num_sims x J 
    for j in range(J):
        if j < 2:
            u_t[:,j] = u_t[:,j] + beta2
        else:
            u_t[:,j] = u_t[:,j] + beta3
            
    numerator = np.exp(u_t) 
    denominator = 1 + np.sum(np.exp(u_t),axis=-1) # num_sims
    
    return np.mean(numerator / (np.tile(denominator, (J, 1)).T), axis=0) 


In [8]:
# 3.2a(iv)
# draw beta coefficients for N individuals S times, observe variation in market share derivatives

S = 100

all_derivatives = np.zeros((J,J,S))
all_shares = np.zeros((J,S))

N = 3000

price = np.array([1,1,1,1])

for s in trange(S):
    beta2 = rng.normal(beta2_bar, sigma2, N)
    beta3 = rng.normal(beta3_bar, sigma3, N)
    all_derivatives[:,:,s] = full_mkt_share_derivative(0, price, beta2, beta3)
    all_shares[:,s] = mkt_share(1, price, beta2, beta3)
(np.mean(all_shares,axis=1), np.std(all_shares,axis=1), np.mean(all_derivatives, axis=2), np.std(all_derivatives, axis=2))

(array([0.04784253, 0.15288083, 0.44046486, 0.35277411]),
 array([0.0008991 , 0.00287306, 0.00211771, 0.0016961 ]),
 array([[-0.29478105,  0.06650959,  0.2168944 ,  0.00709914],
        [ 0.06650959, -0.16315672,  0.09183023,  0.00300568],
        [ 0.2168944 ,  0.09183023, -0.35012373,  0.03100105],
        [ 0.00709914,  0.00300568,  0.03100105, -0.04144621]]),
 array([[3.08401066e-03, 1.64034900e-03, 1.89106999e-03, 6.18963701e-05],
        [1.64034900e-03, 2.14278030e-03, 8.00654110e-04, 2.62061074e-05],
        [1.89106999e-03, 8.00654110e-04, 2.45783477e-03, 3.51894072e-04],
        [6.18963701e-05, 2.62061074e-05, 3.51894072e-04, 2.89493485e-04]]))

In [9]:
mc = np.exp( gamma0 + gamma1*w + omega/8)

In [10]:
# define function to solve

def get_function_to_solve(t, beta2, beta3):
    def F(p):
        # p is a 
        ds_dp = own_mkt_share_derivative(t, p, beta2, beta3)
        shares = mkt_share(t, p, beta2, beta3)
        return p - mc[:,t] + np.reciprocal(ds_dp)*shares
        
    return F


In [11]:
# draw betas, now compute equilibrium prices and shares

beta2 = rng.normal(beta2_bar, sigma2, (N,T))
beta3 = rng.normal(beta3_bar, sigma3, (N,T))

In [12]:


# 3.2 and 3.3: compute equilibrium shares, prices

# these two variables are the prices and shares
eq_prices = np.zeros((J, T))
eq_shares = np.zeros((J, T))

flag_total = 0

for t in trange(T):
    fn = get_function_to_solve(t, beta2[:,t], beta3[:,t])
    mkt_eq_prices, _ , flag, _ = fsolve(fn, np.array([1,1,1,1]), full_output=True)
    flag_total += flag
    eq_prices[:,t] = mkt_eq_prices
    eq_shares[:, t] = mkt_share(t, mkt_eq_prices, beta2[:,t], beta3[:,t])
    
# this should be True iff all of the fsolves converge
flag_total == T

True

In [80]:
# TODO: check that at the equilibrium prices, the estimates for market shares and market share derivatives are precise
#repeating the exercise of simulation with equilibrium prices, trying to get equilibrium shares

S = 100

all_derivatives = np.zeros((J,J,S))
all_shares = np.zeros((J,S))

N = 100

for t in trange(T):
    price = np.array(eq_prices[:,t])
    for s in range(S):
        beta2_s = np.random.normal(beta2_bar, sigma2, N)
        beta3_s = np.random.normal(beta3_bar, sigma3, N)
        all_derivatives[:,:,s] = full_mkt_share_derivative(0, price, beta2_s, beta3_s)
        all_shares[:,s] = mkt_share(t, price, beta2_s, beta3_s)
    
(np.mean(all_shares,axis=1), np.std(all_shares,axis=1), np.mean(all_derivatives, axis=2), np.std(all_derivatives, axis=2))

(array([0.25669534, 0.11349968, 0.39999952, 0.08411012]),
 array([0.01557261, 0.00688554, 0.01892846, 0.00398019]),
 array([[-0.32917796,  0.04810907,  0.19157701,  0.00806251],
        [ 0.04810907, -0.11332643,  0.04445226,  0.00187077],
        [ 0.19157701,  0.04445226, -0.39841075,  0.02477189],
        [ 0.00806251,  0.00187077,  0.02477189, -0.04049647]]),
 array([[1.40807281e-02, 5.50269847e-03, 7.87574330e-03, 3.31450286e-04],
        [5.50269847e-03, 6.92407984e-03, 1.82743524e-03, 7.69075262e-05],
        [7.87574330e-03, 1.82743524e-03, 1.02960062e-02, 1.79170411e-03],
        [3.31450286e-04, 7.69075262e-05, 1.79170411e-03, 1.76051029e-03]]))

In [14]:
# Morrow and Skerlos (2011) Method: (see equation 27 in Conlon + Gortmaker)

def get_matrices(t, p, beta2, beta3):
    # p should be a length J vector
    # betas should be num_sims
    
    u_t = np.tile(x[:,t] + xi[:,t] + alpha*p, (len(beta2), 1)) # num_sims x J 
    for j in range(J):
        if j < 2:
            u_t[:,j] = u_t[:,j] + beta2
        else:
            u_t[:,j] = u_t[:,j] + beta3
            
    Z = np.tile( 1 + np.sum(np.exp(u_t),axis=-1), (J,1)).T # num_sims x J
    
    Lambda_inv = np.zeros((J,J))
    Gamma = np.zeros((J,J))
    
    own_numerator = alpha*np.exp(u_t)  # num_sims x J
    denominator = Z
    
    for j in range(J):
        Lambda_inv[j,j] = 1 / (np.mean(own_numerator / denominator, axis=0)[j])
        
    for j in range(J):
        for k in range(J):
            Gamma[j,k] = np.mean(alpha*np.exp(u_t)[:,k]*np.exp(u_t)[:,j] / np.square(1 + np.sum(np.exp(u_t),axis=-1)))
    
    return Lambda_inv, Gamma

def get_fixed_point_function(t, beta2, beta3):
    ownership_matrix = np.identity(J)
    def F(p):
        Lambda_inv, Gamma = get_matrices(t, p, beta2, beta3)
        shares = mkt_share(t, p, beta2, beta3)
        zeta = np.matmul(np.matmul(Lambda_inv, ownership_matrix*Gamma), (p - mc[:,t])) - np.matmul(Lambda_inv, shares)
        return mc[:,t] + zeta
        
    return F



In [15]:
# Simulate equilibrium using the Morrow and Skerlos (2011) method
eq_prices_2 = np.zeros((J, T))
eq_shares_2 = np.zeros((J, T))

for t in trange(T):
    fn = get_fixed_point_function(t, beta2[:,t], beta3[:,t])
    mkt_eq_prices = fixed_point(fn, np.array([1,1,1,1]), method="iteration")
    eq_prices_2[:,t] = mkt_eq_prices
    eq_shares_2[:, t] = mkt_share(t, mkt_eq_prices, beta2[:,t], beta3[:,t])

# the difference between the two methods, check that this is small
np.max(eq_prices_2 - eq_prices), np.max(eq_shares_2 - eq_shares)

(1.373072322508051e-09, 4.207107107134789e-09)

In [59]:
# Precompute the price elasticities and diversion 
true_price_elasticities = np.zeros((J,J,T))
true_diversion_ratios = np.zeros((J,J,T))

N = 100

for t in trange(T):
    own_price_derivative = own_mkt_share_derivative(t, eq_prices[:,t], beta2[:,t], beta3[:,t])
    derivative_matrix = full_mkt_share_derivative(t, eq_prices[:,t], beta2[:,t], beta3[:,t])
    true_price_elasticities[:,:,t] = eq_prices[:,t]*derivative_matrix / eq_shares[:,t].T
    derivative_matrix = full_mkt_share_derivative(t, eq_prices[:,t], beta2[:,t], beta3[:,t])
    outside_derivatives = outside_mkt_share_derivative(t, eq_prices[:,t], beta2[:,t], beta3[:,t])
    for j in range(J):
        for k in range(J):
            true_diversion_ratios[j,k,t] = -1*derivative_matrix[k,j]/derivative_matrix[j,j]
    for j in range(J):
        true_diversion_ratios[j,j,t] = -1*outside_derivatives[j]/derivative_matrix[j,j]

In [17]:

market_ids = np.tile(np.arange(T) + 1,(J,1)).T.flatten()
firm_ids = np.tile(np.arange(J) + 1,(T,1)).flatten()
satellite = np.concatenate((np.ones((2,T)), np.zeros((2,T)))).T.flatten()
wired = np.concatenate((np.zeros((2,T)), np.ones((2,T)))).T.flatten()
observed_data = pd.DataFrame(data={
    "market_ids": market_ids, 
    "firm_ids": firm_ids,
    "shares": eq_shares.T.flatten(), 
    "prices": eq_prices.T.flatten(),
    "x": x.T.flatten(),
    "satellite": satellite,
    "wired": wired,
    "w": w.T.flatten()
})
unobserved_data = pd.DataFrame(data={
    "market_ids": market_ids, 
    "firm_ids": firm_ids,
    "xi": xi.T.flatten(),
    "omega": omega.T.flatten()
})

In [18]:
# Instrument Analysis

df1 = pd.DataFrame({'p':eq_prices[0,:]})
df1['s'] = pd.Series(eq_shares[0,:])
df1['x'] = pd.Series(x[0,:])
df1['w'] = pd.Series(w[0,:])

df2 = pd.DataFrame({'p':eq_prices[1,:]})
df2['s'] = pd.Series(eq_shares[1,:])
df2['x'] = pd.Series(x[1,:])
df2['w'] = pd.Series(w[1,:])

df3 = pd.DataFrame({'p':eq_prices[2,:]})
df3['s'] = pd.Series(eq_shares[2,:])
df3['x'] = pd.Series(x[2,:])
df3['w'] = pd.Series(w[2,:])


df4 = pd.DataFrame({'p':eq_prices[3,:]})
df4['s'] = pd.Series(eq_shares[3,:])
df4['x'] = pd.Series(x[3,:])
df4['w'] = pd.Series(w[3,:])


modelp11 = smf.ols('p ~ x +w', data=df1)
modelp11 = modelp11.fit()

modelp21 = smf.ols('p ~ x +w', data=df2)
modelp21 = modelp21.fit()

modelp31 = smf.ols('p ~ x +w', data=df3)
modelp31 = modelp31.fit()

modelp41 = smf.ols('p ~ x +w', data=df4)
modelp41 = modelp41.fit()


models11 = smf.ols('s ~ x +w', data=df1)
models11 = models11.fit()

models21 = smf.ols('s ~ x +w', data=df2)
models21 = models21.fit()

models31 = smf.ols('s ~ x +w', data=df3)
models31 = models31.fit()

models41 = smf.ols('s ~ x +w', data=df4)
models41 = models41.fit()

In [19]:
modelp11.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      p   R-squared:                       0.473
Model:                            OLS   Adj. R-squared:                  0.472
Method:                 Least Squares   F-statistic:                     268.4
Date:                Fri, 08 Oct 2021   Prob (F-statistic):           6.91e-84
Time:                        09:31:37   Log-Likelihood:                -77.142
No. Observations:                 600   AIC:                             160.3
Df Residuals:                     597   BIC:                             173.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.3049      0.023     98.211      0.000       2.259       2.351
x              0.1316      0.019      6.886      0.000       0.094       0.169
w              0.4205      0.020     21.362      0.000       0.382       0.459
==============================================================================
Omnibus:                       31.151   Durbin-Watson:                   2.012
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               37.169
Skew:                           0.504   Prob(JB):                     8.49e-09
Kurtosis:                       3.688   Cond. No.                         4.04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Part 4

In [20]:
# 4A: Logit
outside_shares = 1 - np.sum(eq_shares, axis=0, keepdims=True)
y = np.log(eq_shares/outside_shares).T.flatten()
X = observed_data[["x","satellite","wired","prices"]]
results = sm.OLS(y,X).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.314
Model:                            OLS   Adj. R-squared:                  0.313
Method:                 Least Squares   F-statistic:                     365.4
Date:                Fri, 08 Oct 2021   Prob (F-statistic):          2.09e-195
Time:                        09:31:37   Log-Likelihood:                -3033.1
No. Observations:                2400   AIC:                             6074.
Df Residuals:                    2396   BIC:                             6097.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x              0.8375      0.029     28.572      0.000       0.780       0.895
satellite      1.3705      0.122     11.239      0.000       1.131       1.610
wired          1.3589      0.123     11.046      0.000       1.118       1.600
prices        -0.9518      0.044    -21.393      0.000      -1.039      -0.865
==============================================================================
Omnibus:                       41.828   Durbin-Watson:                   2.047
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               48.815
Skew:                          -0.263   Prob(JB):                     2.51e-11
Kurtosis:                       3.460   Cond. No.                         30.0
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Note that ignoring the endogeneity of prices results in underestimating the magnitudes of all the relevant parameters.

# Part 5

## 5.a: Demand-side Estimation only

In [21]:
# BLP, Demand-side estimation only

demand_problem = pyblp.Problem(
    [
        pyblp.Formulation("0 + prices + x + satellite + wired"),
        pyblp.Formulation("0 + satellite + wired"),
        None
    ],
    observed_data,
    integration=pyblp.Integration('product', size=9),
)

Initializing the problem ...
Initialized the problem after 00:00:00.

Dimensions:
 T    N     F     I     K1    K2    MD 
---  ----  ---  -----  ----  ----  ----
600  2400   4   48600   4     2     3  

Formulations:
       Column Indices:             0        1        2        3  
-----------------------------  ---------  -----  ---------  -----
 X1: Linear Characteristics     prices      x    satellite  wired
X2: Nonlinear Characteristics  satellite  wired                  


In [31]:
# we will assume (correctly) that the random coefficients on satellite and wired are uncorrellated

# this step is going to spit out a lot of text, most of which is not meaningful yet. 
# the first iteration of .solve is only to compute the optimal instruments, and hence these first-step estimates are not very good

demand_problem_w_instruments = demand_problem.solve(sigma=np.identity(2)).compute_optimal_instruments().to_problem()

Solving the problem ...

Nonlinear Coefficient Initial Values:
 Sigma:      satellite        wired    
---------  -------------  -------------
satellite  +1.000000E+00               
  wired    +0.000000E+00  +1.000000E+00

Nonlinear Coefficient Lower Bounds:
 Sigma:      satellite        wired    
---------  -------------  -------------
satellite  +0.000000E+00               
  wired    +0.000000E+00  +0.000000E+00

Nonlinear Coefficient Upper Bounds:
 Sigma:      satellite        wired    
---------  -------------  -------------
satellite      +INF                    
  wired    +0.000000E+00      +INF     

Starting optimization ...

GMM   Optimization   Objective   Fixed Point  Contraction  Clipped    Objective      Objective      Projected                                
Step   Iterations   Evaluations  Iterations   Evaluations  Shares       Value       Improvement   Gradient Norm             Theta            
----  ------------  -----------  -----------  -----------  -------  ---

In [32]:
# now we resolve the problem given the optimal instruments
demand_problem_results = demand_problem_w_instruments.solve(sigma=np.identity(2))

Solving the problem ...

Nonlinear Coefficient Initial Values:
 Sigma:      satellite        wired    
---------  -------------  -------------
satellite  +1.000000E+00               
  wired    +0.000000E+00  +1.000000E+00

Nonlinear Coefficient Lower Bounds:
 Sigma:      satellite        wired    
---------  -------------  -------------
satellite  +0.000000E+00               
  wired    +0.000000E+00  +0.000000E+00

Nonlinear Coefficient Upper Bounds:
 Sigma:      satellite        wired    
---------  -------------  -------------
satellite      +INF                    
  wired    +0.000000E+00      +INF     

Starting optimization ...

GMM   Optimization   Objective   Fixed Point  Contraction  Clipped    Objective      Objective      Projected                                
Step   Iterations   Evaluations  Iterations   Evaluations  Shares       Value       Improvement   Gradient Norm             Theta            
----  ------------  -----------  -----------  -----------  -------  ---

These estimates are not bad.

## 5.a: Demand and Supply Estimation

In [33]:
full_problem = pyblp.Problem(
    [
        pyblp.Formulation("0 + prices + x + satellite + wired"),
        pyblp.Formulation("0 + satellite + wired"),
        pyblp.Formulation("1 + w")
    ],
    product_data = observed_data,
    integration=pyblp.Integration('product', size=9),
    costs_type="log"
)

Initializing the problem ...
Initialized the problem after 00:00:00.

Dimensions:
 T    N     F     I     K1    K2    K3    MD    MS 
---  ----  ---  -----  ----  ----  ----  ----  ----
600  2400   4   48600   4     2     2     3     2  

Formulations:
       Column Indices:             0        1        2        3  
-----------------------------  ---------  -----  ---------  -----
 X1: Linear Characteristics     prices      x    satellite  wired
X2: Nonlinear Characteristics  satellite  wired                  
X3: Log Cost Characteristics       1        w                    


In [34]:
# once again, we construct optimal instruments
full_problem_w_instruments = full_problem.solve(sigma=np.identity(2),beta=[-1,None,None,None]).compute_optimal_instruments().to_problem()

Solving the problem ...

Nonlinear Coefficient Initial Values:
 Sigma:      satellite        wired    
---------  -------------  -------------
satellite  +1.000000E+00               
  wired    +0.000000E+00  +1.000000E+00

Beta Initial Values:
   prices            x          satellite        wired    
-------------  -------------  -------------  -------------
-1.000000E+00       NAN            NAN            NAN     

Nonlinear Coefficient Lower Bounds:
 Sigma:      satellite        wired    
---------  -------------  -------------
satellite  +0.000000E+00               
  wired    +0.000000E+00  +0.000000E+00

Beta Lower Bounds:
   prices            x          satellite        wired    
-------------  -------------  -------------  -------------
    -INF           -INF           -INF           -INF     

Nonlinear Coefficient Upper Bounds:
 Sigma:      satellite        wired    
---------  -------------  -------------
satellite      +INF                    
  wired    +0.000000E+00   

In [35]:
# and here are the estimation results
full_problem_results = full_problem_w_instruments.solve(sigma=np.identity(2),beta=[-1,None,None,None])

Solving the problem ...

Nonlinear Coefficient Initial Values:
 Sigma:      satellite        wired    
---------  -------------  -------------
satellite  +1.000000E+00               
  wired    +0.000000E+00  +1.000000E+00

Beta Initial Values:
   prices            x          satellite        wired    
-------------  -------------  -------------  -------------
-1.000000E+00       NAN            NAN            NAN     

Nonlinear Coefficient Lower Bounds:
 Sigma:      satellite        wired    
---------  -------------  -------------
satellite  +0.000000E+00               
  wired    +0.000000E+00  +0.000000E+00

Beta Lower Bounds:
   prices            x          satellite        wired    
-------------  -------------  -------------  -------------
    -INF           -INF           -INF           -INF     

Nonlinear Coefficient Upper Bounds:
 Sigma:      satellite        wired    
---------  -------------  -------------
satellite      +INF                    
  wired    +0.000000E+00   

Computed results after 00:00:21.

Problem Results Summary:
GMM     Objective      Projected    Reduced Hessian  Reduced Hessian  Clipped  Weighting Matrix  Covariance Matrix
Step      Value      Gradient Norm  Min Eigenvalue   Max Eigenvalue   Shares   Condition Number  Condition Number 
----  -------------  -------------  ---------------  ---------------  -------  ----------------  -----------------
 2    +2.933297E+00  +3.130269E-09   +4.288831E+01    +4.496468E+02      0      +6.945432E+16      +2.287690E+04  

Cumulative Statistics:
Computation  Optimizer  Optimization   Objective   Fixed Point  Contraction
   Time      Converged   Iterations   Evaluations  Iterations   Evaluations
-----------  ---------  ------------  -----------  -----------  -----------
 00:01:48       Yes          13           23         128478       392245   

Nonlinear Coefficient Estimates (Robust SEs in Parentheses):
 Sigma:       satellite          wired     
---------  ---------------  ---------------
sat

These estimates are even better than the previous section. We'll use these in the coming sections.

## 5.b Own-price Elasticities, Diversion Ratios

In [36]:
estimated_price_elasticities = full_problem_results.compute_elasticities()

Computing elasticities with respect to prices ...
Finished after 00:00:01.



In [37]:
estimated_diversion_ratios = full_problem_results.compute_diversion_ratios()

Computing diversion ratios with respect to prices ...
Finished after 00:00:01.



In [41]:
estimated_own_price_elasticities = estimated_price_elasticities.reshape(T,J,J).mean(axis=0)

In [60]:
true_price_elasticities.mean(axis=2), estimated_own_price_elasticities

(array([[-4.06535006,  1.38543391,  0.80172334,  0.7895892 ],
        [ 1.27934133, -4.16553436,  0.71112989,  0.71512854],
        [ 0.73928313,  0.74163481, -4.17726162,  1.3416553 ],
        [ 0.72070405,  0.7189693 ,  1.30923805, -4.18978309]]),
 array([[-4.0525488 ,  1.36872713,  0.69795227,  0.66555876],
        [ 1.50752786, -4.15853012,  0.69795227,  0.66555876],
        [ 0.7352649 ,  0.65591371, -4.16252984,  1.39588237],
        [ 0.7352649 ,  0.65591371,  1.4547649 , -4.17736646]]))

The estimates are pretty close to the true values

In [63]:
estimated_diversion_ratios.reshape((T,J,J)).mean(axis=0)

array([[0.32908096, 0.32674409, 0.1743611 , 0.16981386],
       [0.34688774, 0.31879102, 0.16981928, 0.16450196],
       [0.18220138, 0.16573254, 0.32424023, 0.32782585],
       [0.18083009, 0.16332965, 0.33517888, 0.32066138]])

In [64]:
true_diversion_ratios.mean(axis=2)

array([[0.33115087, 0.30335128, 0.18522023, 0.18027762],
       [0.32317153, 0.32122579, 0.18063565, 0.17496703],
       [0.19329289, 0.17575241, 0.32765373, 0.30330097],
       [0.19192008, 0.17341037, 0.31037504, 0.32429451]])

These look reasonably close as well.

# Part 6

In [65]:
# merge firms 1 and 2
observed_data['merger_1_ids'] = observed_data['firm_ids'].replace(2, 1)

# merge firms 1 and 3
observed_data['merger_2_ids'] = observed_data['firm_ids'].replace(3, 1)

In [66]:
marginal_costs = full_problem_results.compute_costs()

merger_1_prices = full_problem_results.compute_prices(
    firm_ids=observed_data['merger_1_ids'],
    costs=marginal_costs
)

merger_2_prices = full_problem_results.compute_prices(
    firm_ids=observed_data['merger_2_ids'],
    costs=marginal_costs
)

Computing marginal costs ...
Finished after 00:00:01.

Solving for equilibrium prices ...
Finished after 00:00:02.

Solving for equilibrium prices ...
Finished after 00:00:02.



In [70]:
np.mean(eq_prices, axis=1)

array([2.73266213, 2.71653207, 2.76078363, 2.73913598])

In [74]:
# relative price changes, merging 1 and 2
np.mean(merger_1_prices.reshape((T,J)),axis=0)  

array([2.98076192, 2.99491833, 2.77124866, 2.74875183])

In [68]:
# relative price changes, merging 1 and 3
np.mean(merger_2_prices.reshape((T,J)),axis=0)

array([2.84644941, 2.72849847, 2.88261994, 2.75138542])

In [71]:
reduction_factors = np.concatenate([0.85*np.ones([T,2]),np.ones([T,2])],axis=1).reshape((T*J,1))
reduced_costs = marginal_costs * reduction_factors


merger_1_prices_w_cost_reduction = full_problem_results.compute_prices(
    firm_ids=observed_data['merger_1_ids'],
    costs=reduced_costs
)


Solving for equilibrium prices ...
Finished after 00:00:03.



In [72]:
# post-merger relative price changes, 1 and 2 with marginal cost reduction
np.mean(merger_1_prices_w_cost_reduction.reshape((T,J)),axis=0)

array([2.78332681, 2.79535976, 2.76117594, 2.73907293])

In [75]:
pre_merger_surpluses = full_problem_results.compute_consumer_surpluses()
post_merger_surpluses = full_problem_results.compute_consumer_surpluses(prices=merger_1_prices_w_cost_reduction)

Computing consumer surpluses with the equation that assumes away nonlinear income effects ...
Finished after 00:00:01.

Computing consumer surpluses with the equation that assumes away nonlinear income effects ...
Finished after 00:00:00.



In [76]:
# assuming measure of consumers in each market is 1, the net surpluses are just the sums
# this is the net effect on consumer welfare
np.sum(post_merger_surpluses - pre_merger_surpluses)

-6.838416907825487

In [77]:
post_merger_shares = full_problem_results.compute_shares(merger_1_prices_w_cost_reduction)
pre_merger_profits = full_problem_results.compute_profits()
post_merger_profits = full_problem_results.compute_profits(merger_1_prices_w_cost_reduction, post_merger_shares, reduced_costs)

Computing shares ...
Finished after 00:00:00.

Computing profits ...
Finished after 00:00:01.

Computing profits ...
Finished after 00:00:00.



In [78]:
# once again assuming measure 1 of consumers in each market
# net change in profits
np.sum(post_merger_profits - pre_merger_profits) 

69.32302854307048

In [79]:
# welfare change
np.sum(post_merger_surpluses - pre_merger_surpluses) + np.sum(post_merger_profits - pre_merger_profits) 

62.48461163524499